# Redshift Cluster & Database Operations   

In [1]:
import boto3
import configparser
import matplotlib.pyplot as plt
import pandas as pd
from time import time
import json

In [2]:
config = configparser.ConfigParser()
config.read_file(open('aws.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
CLUSTER_REGION        = config.get('AWS','CLUSTER_REGION')


ec2 = boto3.resource('ec2',
                       region_name=CLUSTER_REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',
                       region_name=CLUSTER_REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

## Just a pretty print
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [3]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

DWH_CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","CLUSTER_IDENTIFIER")
DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "IAM_ROLE_NAME")

DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")


## Create IAM Role

In [5]:
try:
    print('1.1 Creating a new IAM Role')
    
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'}))

except Exception as e:
    print(e)

# TODO: Attach Policy
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

1.1 Creating a new IAM Role
1.2 Attaching Policy


200

## Create Redshift Cluster

In [6]:
# TODO: Get and print the IAM role ARN
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

1.3 Get the IAM role ARN
arn:aws:iam::147551276302:role/dwhRole


## Get Redshift Cluster Endpoint

In [13]:
# FILL IN THE REDSHIFT ENPOINT HERE
# e.g. DWH_ENDPOINT="redshift-cluster-1.csmamz5zxmle.us-west-2.redshift.amazonaws.com" 
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
    
#FILL IN THE IAM ROLE ARN you got in step 2.2 of the previous exercise
#e.g DWH_ROLE_ARN="arn:aws:iam::988332130976:role/dwhRole"

DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cmfoxim90hks.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::147551276302:role/dwhRole


## Describe Cluster

In [14]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\josep\AppData\Local\Temp\ipykernel_24276\2744060160.py:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,pausing
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cmfoxim90hks.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-004b48aa49630125b
7,NumberOfNodes,4


## Open incoming TCP port from the VPC to access the cluster endpoint

In [10]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,  # TODO: fill out
        CidrIp= '0.0.0.0/0',  # TODO: fill out
        IpProtocol= 'TCP',  # TODO: fill out
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-06de6078a36306d85')


## Pause Cluster

In [12]:
####   Pause cluster
response = redshift.pause_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)
prettyRedshiftProps(response)

C:\Users\josep\AppData\Local\Temp\ipykernel_24276\2744060160.py:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value


## Resume Cluster

In [5]:
response = redshift.resume_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)
prettyRedshiftProps(response)

C:\Users\josep\AppData\Local\Temp\ipykernel_24276\2744060160.py:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value


## Delete Cluster

In [ ]:
#-- Uncomment & run to delete the created resources
response = redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
prettyRedshiftProps(response)
#### CAREFUL!!